In [44]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten,Conv2D,Conv1D,MaxPooling1D,  Dropout
from keras.layers import concatenate
from keras.layers import Bidirectional, GlobalMaxPool1D,BatchNormalization
from keras.models import Model, Sequential
from keras.layers import Convolution1D,GlobalMaxPooling1D
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.utils import to_categorical
import numpy as np
import sys
import os

In [2]:
df = pd.read_csv("C:/Users/Harvey/Desktop/Yelp_data_set/restuarant_review_5_label_unbalanced.csv")
df['Processed_Reviews'][0]
df.head()

,stars,Processed_Reviews
0,5,go lunch steak sandwich delicious caesar salad...
1,4,ill first admit excite go la tavolta food snob...
2,3,tracy dessert big name hong kong one first mar...
3,1,place go hill clearly cut back staff food qual...
4,4,like walk back time every saturday morning sis...


In [3]:
train = df.sample(frac = 0.8,random_state = 200)
test = df.drop(train.index)

In [4]:
train.loc[:,'stars'] -= 1
print(np.unique(train['stars']))

[0 1 2 3 4]


In [7]:
print(train.head())

        stars                                  Processed_Reviews
99837       0  waiter condescend jerk ive ever encounter girl...
343500      4  oh place addictive ive hear many good things i...
168738      4  great food great atmosphere accommodate highly...
198136      4  first time eat pastolis great experience basil...
309110      4  great happy hour price always bring compliment...


In [57]:
max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train['Processed_Reviews'])
list_tokenized_train = tokenizer.texts_to_sequences(train['Processed_Reviews'])


maxlen = 130
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = to_categorical(train['stars'])
#####################
# Test data
tokenizer.fit_on_texts(test['Processed_Reviews'])
list_tokenized_test = tokenizer.texts_to_sequences(test['Processed_Reviews'])


maxlen = 130
X_test = pad_sequences(list_tokenized_test, maxlen=maxlen)
test.loc[:,'stars'] -=1
y_test = test['stars']
print(np.unique(test['stars']))

[0 1 2 3 4]


In [6]:
print(y[:5,])
print(train['stars'][:5])

[[1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]
99837     0
343500    4
168738    4
198136    4
309110    4
Name: stars, dtype: int64


In [50]:
embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size,input_length = maxlen))
model.add(Conv1D(filters = 50, kernel_size = 2, padding='same',activation = "relu"))
model.add(GlobalMaxPool1D())
#model.add(Flatten())
model.add(Dense(5, activation="softmax"))
model.summary()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 130, 128)          768000    
_________________________________________________________________
conv1d_32 (Conv1D)           (None, 130, 50)           12850     
_________________________________________________________________
global_max_pooling1d_11 (Glo (None, 50)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 5)                 255       
Total params: 781,105
Trainable params: 781,105
Non-trainable params: 0
_________________________________________________________________


In [51]:
batch_size = 100
epochs = 3
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 377894 samples, validate on 94474 samples
Epoch 1/3
377894/377894 [==============================] - 193s 511us/step - loss: 0.8832 - acc: 0.6172 - val_loss: 0.8300 - val_acc: 0.6398
Epoch 2/3
377894/377894 [==============================] - 189s 499us/step - loss: 0.7870 - acc: 0.6586 - val_loss: 0.8193 - val_acc: 0.6441
Epoch 3/3
377894/377894 [==============================] - 190s 503us/step - loss: 0.7449 - acc: 0.6785 - val_loss: 0.8282 - val_acc: 0.6439


In [58]:
prediction = model.predict(X_test)
y_pred = np.argmax(prediction,axis = 1)
y_test = np.array(y_test)
print(y_pred[:10])
print(y_test.shape)
print(y_pred.shape)
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, confusion_matrix
print('accuracy :{0}'.format(accuracy_score(y_pred, y_test)))

[3 2 4 3 4 4 4 3 3 2]
(118092,)
(118092,)
accuracy :0.43789587779019745


In [16]:

#####################################################################
# Using pretrained glove vector
#####################################################################
GLOVE_DIR = "C:/Users/Harvey/Desktop/Yelp_data_set/"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'),encoding = 'utf8')

In [17]:
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [18]:
print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))

Total 400000 word vectors in Glove 6B 100d.


In [22]:
word_index = tokenizer.word_index
embed_size = 100
embedding_matrix = np.random.random((len(word_index) + 1, embed_size))

In [23]:

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
embedding_layer = Embedding(len(word_index) + 1,
                            embed_size,
                            weights=[embedding_matrix],
                            input_length=maxlen,
                            trainable=False)

In [24]:
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(filters = 50, kernel_size = 2, padding='same',activation = "relu"))
model.add(GlobalMaxPool1D())
#model.add(Flatten())
model.add(Dense(5, activation="softmax"))
model.summary()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 130, 100)          23769300  
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 130, 50)           10050     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 255       
Total params: 23,779,605
Trainable params: 10,305
Non-trainable params: 23,769,300
_________________________________________________________________


In [25]:
batch_size = 100
epochs = 3
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 377894 samples, validate on 94474 samples
Epoch 1/3
377894/377894 [==============================] - 130s 345us/step - loss: 0.9773 - acc: 0.5789 - val_loss: 0.9014 - val_acc: 0.6104
Epoch 2/3
377894/377894 [==============================] - 131s 348us/step - loss: 0.8879 - acc: 0.6153 - val_loss: 0.8843 - val_acc: 0.6182
Epoch 3/3
377894/377894 [==============================] - 131s 346us/step - loss: 0.8695 - acc: 0.6223 - val_loss: 0.8759 - val_acc: 0.6214


In [26]:
#########################################
# Yoon kim architure
embedding_layer = Embedding(len(word_index) + 1,
                            embed_size,
                            weights=[embedding_matrix],
                            input_length=maxlen,
                            trainable=True)

In [27]:
convs = []
filter_sizes = [3,4,5]

sequence_input = Input(shape=(maxlen,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

In [36]:
for fsz in filter_sizes:
    l_conv = Conv1D(nb_filter=128,filter_length=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    convs.append(l_pool)
   
l_merge = concatenate(convs,axis=1) 
l_cov1= Conv1D(128, 5, activation='relu')(l_merge)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = GlobalMaxPool1D()(l_cov2)
#l_flat = Flatten()(l_pool2)
l_dense = Dense(128, activation='relu')(l_pool2)
preds = Dense(5, activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

C:\Users\Harvey\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=3)`
  
C:\Users\Harvey\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=4)`
  
C:\Users\Harvey\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=5)`
  


In [37]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 130)          0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 130, 100)     23769300    input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 128, 128)     38528       embedding_4[0][0]                
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 127, 128)     51328       embedding_4[0][0]                
__________________________________________________________________________________________________
conv1d_7 (

In [38]:
batch_size = 100
epochs = 3
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 377894 samples, validate on 94474 samples
Epoch 1/3
377894/377894 [==============================] - 4897s 13ms/step - loss: 0.9161 - acc: 0.6011 - val_loss: 0.8522 - val_acc: 0.6283
Epoch 2/3
 25100/377894 [>.............................] - ETA: 17:23:13 - loss: 0.8363 - acc: 0.6331

KeyboardInterrupt: 

In [ ]:
############################
# Testing 
#############################
prediction = model.predict(X_test)
y_pred = np.argmax(prediction,axis = 1)
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, confusion_matrix
print('accuracy :{0}'.format(accuracy_score(y_pred, y_test)))
print('F1-score: {0}'.format(f1_score(y_pred, y_test)))

128


In [70]:
embed_size = 100
embedding_layer = Embedding(len(word_index) + 1,
                            embed_size,
                            weights=[embedding_matrix],
                            input_length=maxlen,
                            trainable=False)

In [71]:
#############################################
# Original yoon kim 
#############################################
conv_filters = 128
sequence_input = Input(shape=(maxlen,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

# Specify each convolution layer and their kernel siz i.e. n-grams 
conv1_1 = Conv1D(filters=conv_filters, kernel_size=3)(embedded_sequences)
btch1_1 = BatchNormalization()(conv1_1)
actv1_1 = Activation('relu')(btch1_1)
drp1_1  = Dropout(0.2)(actv1_1)
glmp1_1 = GlobalMaxPooling1D()(drp1_1)

conv1_2 = Conv1D(filters=conv_filters, kernel_size=4)(embedded_sequences)
btch1_2 = BatchNormalization()(conv1_2)
actv1_2 = Activation('relu')(btch1_2)
drp1_2  = Dropout(0.2)(actv1_2)
glmp1_2 = GlobalMaxPooling1D()(drp1_2)

conv1_3 = Conv1D(filters=conv_filters, kernel_size=5)(embedded_sequences)
btch1_3 = BatchNormalization()(conv1_3)
actv1_3 = Activation('relu')(btch1_3)
drp1_3  = Dropout(0.2)(actv1_3)
glmp1_3 = GlobalMaxPooling1D()(drp1_3)

conv1_4 = Conv1D(filters=conv_filters, kernel_size=6)(embedded_sequences)
btch1_4 = BatchNormalization()(conv1_4)
actv1_4 = Activation('relu')(btch1_4)
drp1_4  = Dropout(0.2)(actv1_4)
glmp1_4 = GlobalMaxPooling1D()(drp1_4)

# Gather all convolution layers
cnct = concatenate([glmp1_1, glmp1_2, glmp1_3, glmp1_4], axis=1)
drp1 = Dropout(0.2)(cnct)

dns1  = Dense(32, activation='relu')(drp1)
btch1 = BatchNormalization()(dns1)
drp2  = Dropout(0.2)(btch1)

out = Dense(5, activation='softmax')(drp2)

In [72]:
model = Model(inputs=sequence_input, outputs=out)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 130)          0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 130, 100)     23769300    input_8[0][0]                    
__________________________________________________________________________________________________
conv1d_33 (Conv1D)              (None, 128, 128)     38528       embedding_8[0][0]                
__________________________________________________________________________________________________
conv1d_34 (Conv1D)              (None, 127, 128)     51328       embedding_8[0][0]                
__________________________________________________________________________________________________
conv1d_35 

In [ ]:
batch_size = 100
epochs = 3
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 377894 samples, validate on 94474 samples
Epoch 1/3
377894/377894 [==============================] - 2175s 6ms/step - loss: 1.0041 - acc: 0.5680 - val_loss: 0.8962 - val_acc: 0.6113
Epoch 2/3
113700/377894 [========>.....................] - ETA: 1:35:23 - loss: 0.9160 - acc: 0.6044

In [ ]:
prediction = model.predict(X_test)
print(y_pred.shape)
y_pred = np.argmax(prediction,axis = 1)
y_test = np.array(y_test)
print(y_pred[:10])
print(y_test.shape)
print(y_pred.shape)
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, confusion_matrix
print('accuracy :{0}'.format(accuracy_score(y_pred, y_test)))